In [1]:
# load libraries
from datetime import datetime, time, timedelta
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import numpy as np
import pandas as pd
import datetime as dt
import plotly
import plotly.graph_objs as go

init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings("ignore")

In [2]:
def detalized_graphs(data, entity, x, y, title):
    
    data_for_plot = []

    for ent in set(data[entity]):

        entity_data = data[data[entity] == ent]
        trace = go.Scatter(x=entity_data[x],y=entity_data[y], name = ent)
        data_for_plot.append(trace)

    # Set the title
    layout = {'title': title}

    # Create a Figure and plot it
    fig = go.Figure(data=data_for_plot, layout=layout)
    iplot(fig, show_link=False)

**Задача:**
1. Воспроизведите график числа рекламных событий по дням. 
2. Найдите причину резкого увеличения количества рекламных событий и объясните, что произошло. 
3. Предложите способ, который помог бы автоматически детектировать аномальные изменения метрик на графиках рекламной статистики. Иными словами, предложите алгоритм, который каждое утро анализирует данные за прошедшие сутки, и, если они сильно выбиваются из общего тренда, отправляет аналитику уведомление: на этом графике есть проблема за вчерашний день. Подумайте, как сделать детектор, который проверяет на аномалии каждый час, каждые пять минут в графике? 
Предложите вариант кода, который решает эту задачу. Реализовывать логику нотификации не нужно, только поиск аномального дня, часа, пятиминутки в данных рекламной статистики. 

Немного дополнительной информации о структуре рекламы ВКонтакте. Есть две основные группы рекламодателей: агентства и клиенты. Агентствами являются рекламные кабинеты юридических лиц. Среди клиентов же существуют как клиенты этих самых агентств, так и автономные рекламные кабинеты физических лиц, над которыми нет агентств. Рекламные кампании — это наборы рекламных объявлений. Они есть в каждом кабинете. 


In [3]:
data = pd.read_csv('test_data.csv')

** Значения полей**  
* time — время взаимодействия пользователя с рекламой в формате UnixTime;
* event — тип взаимодействия (click — клик на объявление, view — показ объявления, report — жалоба на объявление);
* ad_cost_type — тип трат рекламного объявления (CPM — траты за 1000 показов, CPC — траты за клик);
* has_video — наличие видео в объявлении (0 — нет, 1 — есть);
* agency_union_id — id рекламного агентства (если рекламный кабинет принадлежит физическому лицу, то ячейка будет заполнена значением 'no_agency');
* client_union_id — id рекламного клиента;
* campaign_union_id — id рекламной кампании;
* ad_id — id рекламного объявления;
* platform — платформа, с которой пользователь взаимодействовал с рекламой (web — веб-версия, mvk — мобильная версия, * iphone — IOS-приложение, android — Android-приложение, wphone — Windows Phone-приложение).

In [4]:
data['event'].value_counts() # Из общего числа логов по каждому типу действия, выбивается количество показов объявлений

view      87975
click     28639
report     5483
Name: event, dtype: int64

In [5]:
data.head()

,time,event,ad_cost_type,has_video,agency_union_id,client_union_id,campaign_union_id,ad_id,platform,date
0,1.554744e+09,report,CPM,0,no_agency,client_21074,campaign_27489,ad_27489,android,2019-04-08
1,1.555103e+09,click,CPM,0,no_agency,client_22392,campaign_35893,ad_35893,android,2019-04-12
2,1.555403e+09,click,CPM,0,no_agency,client_16836,campaign_26799,ad_26804,web,2019-04-16
3,1.554822e+09,click,CPM,0,no_agency,client_22920,campaign_37146,ad_37146,android,2019-04-09
4,1.554967e+09,click,CPM,0,no_agency,client_645,campaign_15893,ad_15944,web,2019-04-11


In [6]:
data['time'] = data['time'].apply(lambda x: dt.datetime.fromtimestamp(x))

In [7]:
df = data.groupby(['date','event']).size().reset_index(name = 'n_events')

In [8]:
detalized_graphs(df, 'event', 'date', 'n_events', 'Динамика количества действий с рекламными объявлениями')

In [9]:
apr11_views = data[(data['date'] == '2019-04-11')&(data['event'] == 'view')]

In [10]:
print("Статистика по показам объявлений на 11 апреля")
apr11_views.groupby('ad_id').size().describe()

Статистика по показам объявлений на 11 апреля


count     879.000000
mean       18.312856
std       345.487403
min         1.000000
25%         1.000000
50%         1.000000
75%         2.000000
max      9631.000000
dtype: float64

Мы видим, что самое часто просматриваемое объявление было показано 11 апреля 9631 раз, в то время, как 75% объявлений было показано не более 2 раз.

In [11]:
print("Топ-10 самых просматриваемых объявлений 11 апреля")
apr11_views.groupby('ad_id').size().sort_values(ascending = False).head(10)

Топ-10 самых просматриваемых объявлений 11 апреля


ad_id
ad_49554    9631
ad_49556    3454
ad_49560     512
ad_49564     334
ad_388       101
ad_47269      79
ad_99303      69
ad_387        42
ad_1181       38
ad_3021       26
dtype: int64

In [12]:
apr11_views[apr11_views['ad_id'].isin(['ad_49554','ad_49556','ad_49560','ad_49564'])].drop_duplicates('ad_id')

,time,event,ad_cost_type,has_video,agency_union_id,client_union_id,campaign_union_id,ad_id,platform,date
19,2019-04-11 22:30:01,view,CPC,0,agency_2,client_47270,campaign_49554,ad_49554,android,2019-04-11
25,2019-04-11 09:31:51,view,CPC,0,agency_2,client_47270,campaign_49554,ad_49560,android,2019-04-11
47,2019-04-11 17:42:32,view,CPC,0,agency_2,client_47270,campaign_49554,ad_49556,android,2019-04-11
115,2019-04-11 19:45:41,view,CPC,0,agency_2,client_47270,campaign_49554,ad_49564,iphone,2019-04-11


Первые 4 объявления (а особенно, топ-2) явно выбиваются и все они принадлежат одному рекламному агенству 'agency_2' и клиенту, и одной и той же рекламной кампании. Посмотрим детальнее на статистику по этой рекламной компании. 

In [13]:
suspected_ads_data = data[(data['event'] == 'view')& (data['campaign_union_id'] == 'campaign_49554')].groupby(['date', 'ad_id']).size().reset_index(name='views')

In [14]:
detalized_graphs(suspected_ads_data, 'ad_id', 'date', 'views', 'Динамика количества показов объявлений рекламной кампании с id 49554')

Рассмотрим детальнее аномальное объявление - ad_49554

In [15]:
suspected_ads_data = data[(data['event'] == 'view')& (data['ad_id'] == 'ad_49554')].groupby(['date', 'platform']).size().reset_index(name='views')

In [16]:
detalized_graphs(suspected_ads_data, 'platform', 'date', 'views', 'Динамика количества показов объявления 49554 в разбивке по типу платформы пользователей')

Не труднозаметить, что показы на платформе Android явно выбиваются

In [17]:
suspected_ads_data = data[(data['ad_id'] == 'ad_49554') & (data['event'] == 'view') &(data['platform'] == 'android')]

In [18]:
suspected_ads_data['ad_cost_type'].unique()

array(['CPC'], dtype=object)

In [19]:
suspected_ads_data['has_video'].unique()

array([0])

Агент платил за данное рекламное объявление по принципу "траты за клик" и внем не было видео.

In [20]:
suspected_ads_data['time'] = suspected_ads_data['time'].apply(lambda x: dt.datetime.strftime(x, "%H"))
suspected_ads_data = suspected_ads_data.groupby(['date','time']).size().reset_index(name = 'counts')

detalized_graphs(suspected_ads_data, 'date', 'time', 'counts', 'Динамика количества показов объявления 49554 на платформе Android по часам')

**Аномальное увеличение количества показов объявления 49554 произошло 11-го апреля в 13 часов, но с 14 до 19 часов тренд постепенно снижался. Однозначно назвать причину такого увеличения, имея только эти данные, сложно. Так как у нас нет уникальных id пользователей, просмотревших объявление, нельзя подтвердить гипотезу о дубликатах в логах. Для проверки данной гипотезы стоит выгрузить больше детализированных данных (с местами показов и пользователями) по просмотрам рекламы с id 49554 на платформе Android 11 апреля с 13 по 14 часов.Стоит также проверить настройки самой рекламы и активность пользователей VK в этот период. **

## Способ, автоматической детектиции аномальных изменений метрик рекламной статистики.

Один из наиболее удобных способов автоматического детектирования аномальных изменений метрик рекламной статистики - настройка алертинга в BI-системах (н-р, в Tableau). Для этого нужно настроить real-time дашборд и в случае выполнения определенного условия (н-р, превышения метрикой определенной отметки), на почту пользователям дашборда будет приходить уведомление.

Другой способ - алгоритм, анализирующий статистику за конкретный интервал времени. Поскольку наданном графике сезонности по дням недели не наблюдается, за правило нотификации можем брать рост метрики более, чем на определенное количество % по сравнению со средним этой же метрики за предыдущие периоды. При наличии эффекта сезонности в данных (н-р по дням недели) это обязательно нужно это учитывать. Поскольку на данном графике сезонности не наблюдается, за правило возьмем среднее за 3 предыдущих дня. Однако это правило гибкое (его можно пересмотреть).

In [21]:
# Алгоритм (функция проверки дня на наличие аномалий)

def analyze_stats_per_interval(data, # анализируемые данные (должны иметь поле time - дата и в ремя в стандартном формате)
                                     # должны включать в себя данные за 3 дня до анализируемого дня
                               date, # анализируемый день (Если сегодня анализируем данные за вчерашний день, 
                                     # в поле date будет передаваться вчерашняя дата )
                               min_interval, # анализируемый интервал времени в минутах
                               anomaly_increase): # число в процентах, которое формирует 
                                                  # правило (уведомлять, если метрика отклонилась от нормы на столько %)
    date_0 = pd.to_datetime(date)
    date_1 = date_0 + timedelta(1)
    date_2 = date_0 - timedelta(3)
    daily_data = data[(data['time'] >= date_0) & (data['time'] < date_1)]
    daily_data.set_index('time', inplace = True)
    daily_data['count'] = 1
    rule = str(min_interval)+'T'
    agreg_data =  daily_data.resample(rule, label = 'left', closed = 'left', how = sum).bfill().reset_index()
    interv_data = agreg_data['count'].tolist()
    
    three_days_before_data = data[(data['time'] >= date_2) & (data['time'] < date_0)]
    three_days_before_data.set_index('time', inplace = True)
    three_days_before_data['count'] = 1
    agreg_data_3days =  three_days_before_data.resample(rule, label = 'left', closed = 'left', how = sum).bfill().reset_index()
    agreg_data_3days['time'] = agreg_data_3days['time'].apply(lambda x: dt.datetime.strftime(x, '%H:%M:%S'))
    
    three_days_mean = agreg_data_3days.groupby('time').mean()['count'].tolist()
    
    anomaly_increase = anomaly_increase/100 + 1

    benchmark_metric = [i*anomaly_increase for i in three_days_mean]
    
    probs = []
    
    for i in range(len(interv_data)):
        if interv_data[i]>benchmark_metric[i]:
            probs.append(dt.datetime.strftime(agreg_data['time'][i], '%H:%M:%S'))
            
    
    return print("There is anomalies in metric starting from {} with {} minutes interval".format(probs, min_interval))
 

In [22]:
analyze_stats_per_interval(data,'2019-04-11',60, 40) # Пример

There is anomalies in metric starting from ['12:00:00', '13:00:00', '14:00:00', '15:00:00', '16:00:00', '17:00:00', '18:00:00', '19:00:00', '20:00:00', '21:00:00', '22:00:00', '23:00:00'] with 60 minutes interval


Если нужно проверять метрику каждые 5 минут, нужно в немного кастомизировать функцию и добавить туда ограничение не только по дате, но и по времени, и уже после настроить автозапуск функции на продакшене с помощью докера.